In [1]:
import time
import sys
# from graph import graph # my own code for graph object
import heapq
from collections import deque,defaultdict
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable # for linear programming

In [2]:
from collections import deque,defaultdict
class vertex:
    def __init__(self,node):
        self.id = node
        self.adjacent = set()

    def __str__(self):
        # for print out result
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])

    def add_neighbor(self, neighbor):
    
        self.adjacent.add(neighbor)

    def remove_neighbor(self, neighbor):
        if neighbor in self.adjacent:
            self.adjacent.remove(neighbor)

    def is_connected(self,neighbor):
        return neighbor in self.adjacent

    def get_connections(self):
        return self.adjacent



class graph:
    # unweighted undirected graph
    # can be connected or not
    def __init__(self):
        self.vert_dict = {} # vertex_id (int) : vertex
        self.num_vertices = 0
        self.num_edges = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self,node):
        self.num_vertices += 1
        new_vertex = vertex(node)
        self.vert_dict[node] = new_vertex

    def get_vertex(self,node):
        if node in self.vert_dict:
            return self.vert_dict[node]
        else:
            return None

    def add_edge(self, frm, to):
        # for new vertices
        if frm not in self.vert_dict:
            self.add_vertex(frm)
        if to not in self.vert_dict:
            self.add_vertex(to)

        if not self.vert_dict[frm].is_connected(self.vert_dict[to]):
            self.num_edges += 1

        self.vert_dict[frm].add_neighbor(self.vert_dict[to])
        self.vert_dict[to].add_neighbor(self.vert_dict[frm])


    def remove_edge(self, frm, to):
        self.vert_dict[frm].remove_neighbor(self.vert_dict[to])
        self.vert_dict[to].remove_neighbor(self.vert_dict[frm])
        self.num_edges -= 1
    
    def remove_vertex(self,node,inplace = False):
        
        G = self.copy() if not inplace else self
        
        for neighbor in list(G.vert_dict[node].get_connections()):
            G.remove_edge(node, neighbor.id)
            if not neighbor.get_connections():
                del G.vert_dict[neighbor.id]
        del G.vert_dict[node]
        
        return G
            

    def get_vertices(self):
        # return a list of ints, the id of vertices
        return list(self.vert_dict.keys())
    
    def get_vertex_degree(self,node):
        return len(self.get_vertex(node).adjacent)
    
    def get_max_degree_vertex(self):
        return max(self.vert_dict, key = lambda x: len(self.get_vertex(x).get_connections()))
    
    def copy(self):
        copy = graph()
        for v in self.vert_dict:
            for u in self.vert_dict[v].get_connections():
                copy.add_edge(v,u.id)
        return copy


In [671]:
G.get_max_degree_vertex()

34

In [3]:
# read data and construct a graph object
def parse_edges(filename):
    # parse edges from graph file to create your graph object
    # filename: string of the filename
    f = open(filename, "r")
    n_vertices, n_edges, _ = f.readline().split(' ')
    n_vertices, n_edges = int(n_vertices), int(n_edges)

    G = graph() # create a graph

    # add edges to the graph
    for i in range(1,n_vertices+1):
        newline = f.readline()
        newline = newline.rstrip('\n')
        newline = newline.strip()
        
        if not newline:
            G.add_vertex(i)
        else:
            neighbors = list(map(int, newline.split(' ')))

            for neighbor in neighbors:
                if neighbor > i:
                    G.add_edge(i, neighbor)
                
    return G  

In [4]:
# G = parse_edges('../DATA/karate.graph')
# G = parse_edges('../DATA/netscience.graph')
G = parse_edges('../DATA/dummy2.graph')

In [626]:
for v in G:
    print(v)

1 adjacent: [2, 3]
2 adjacent: [4, 1, 3]
3 adjacent: [2, 1, 5]
4 adjacent: [2, 6, 5]
5 adjacent: [4, 3]
6 adjacent: [4]


In [619]:
G1 = G.remove_vertex(4)

## 2-approximation Algorithm to find a lower bound

In [5]:
def approx2(G):
    # G: a graph object
    # return: the number of vertices of a 2-approximation solution to the min vertex cover problem of G
    S = set()
    for v in G:
        if v in S:
            continue
        for neighbor in v.get_connections():
            
            if neighbor.id < v.id:
                continue
            if v.id not in S and neighbor.id not in S:
                S.add(v.id)
                S.add(neighbor.id)
    return len(S)

Since $|S| \leq 2 OPT(G)$, we know $|S|/2$ is a lower bound of $OPT(G)$.

In [6]:
LB = approx2(G) // 2
n = G.num_vertices

In [7]:
LB

3

## Branch-and-Bound

$C'$: partial vertex cover of $G = (V, E)$

$G' = (V', E')$: subgraph not covered by $C'$

$V' = V - C' - \{u | \forall (u,v) \in E,  v \in C'\}$

$E' = \{(u,v) \in E | u, v \in V' \}$

LB = $|C'|$ + LB of the VC for $G'$

* If we have $V'$ empty, we found a vertex cover
* LB of the VC for $G'$ can be found using approx2($G'$)
* The problem is how to do recursion and how to find and update $G'$

In [8]:
def Find_Vprime(G, Cprime):
    'Cprime: a list of numbers in {1, 2, ..., n}'
    'Return: a set of vertices Vprime of the graph Gprime'
    Vprime = set()
#     Cprime = set(Cprime)
    n = G.num_vertices
    for node in range(1, n+1):
        if node in Cprime:
            continue
            
        for neighbor in G.get_vertex(node).adjacent:
            if neighbor.id not in Cprime:
                Vprime.add(node)
                break
    return Vprime

In [9]:
def Find_LowerBound(V_prime):
    'Find lower bound by linear programming, slow'
    'Cprime: a list of vertices of a vertex cover'
   
    V_prime = {i:x for i,x in zip(V_prime, range(len(V_prime)))} # nodes: variable index
    
    # Prepare for Linear Regression
    obj = [1] * len(V_prime) # coeff of objective function
    bnd = [(0,1)] * len(V_prime) # bounds for each variable

    # Linear constraints, has to be <= 
    lhs_ineq = []
    rhs_ineq = []

    for node in V_prime:
        for neighbor in G.get_vertex(node).adjacent:
            if neighbor.id > node and neighbor.id in V_prime:
                new_constraint = [0]*len(V_prime)
                new_constraint[V_prime[node]] = -1
                new_constraint[V_prime[neighbor.id]] = -1
                lhs_ineq.append(new_constraint)
                rhs_ineq.append(-1)

    # solve the linear programming problem
    print('Number of constraints:', len(lhs_ineq))
    opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd,
                  method="revised simplex")
    
    # lowerbound is half of the optimal of the linear optimization problem
    lowerbound = opt.fun // 2
    
    return lowerbound

In [10]:
def Gprime(G, V_prime):
    Gprime = graph()
    for node in V_prime:
        for neighbor in G.get_vertex(node).adjacent:
            if neighbor.id > node and neighbor.id in V_prime:
                Gprime.add_edge(node,neighbor.id)
    return Gprime

In [11]:
G = parse_edges('../DATA/karate.graph')

In [679]:
## This code is working, but not very efficient, trying to improve the speed using the following cell
class node:
    def __init__(self,id, parent,lb,state):
        self.id = id
        self.parent = parent
        self.lb = lb
        self.state = state
#         self.degree = degree
    
    def __lt__(self, other):
        return self.lb > other.lb
        
# dummy = node(0,None,LB)       

n = G.num_vertices
opt_num = n
vertices = G.get_vertices()
vertices.sort(key = lambda x: -len(G.get_vertex(x).adjacent))
opt_cover = vertices
LB = approx2(G) / 2
vertices_order = {vertex: i for i, vertex in enumerate(vertices)}

num_uncov_edges0 = G.num_edges
num_uncov_edges1 = G.num_edges - G.get_vertex_degree(vertices[0])
pqueue = [(num_uncov_edges0, node(vertices[0],None,LB,0)),(num_uncov_edges1,node(vertices[0],None,LB,1))]
heapq.heapify(pqueue)

while pqueue:

    num_uncov_edges,Dnode = heapq.heappop(pqueue)
    parent = Dnode.parent
    node_id = Dnode.id   
        
        
    Cprime = set([Dnode.id]) if Dnode.state else set()
    explored = set() if Dnode.state else set([Dnode.id])
    while parent:
#         print([parent.id, parent.state],end='')
        if parent.state:
            Cprime.add(parent.id)
        else:
            explored.add(parent.id)
        parent = parent.parent
#     print('')
#     print('Cprime',Cprime) 
    V_prime = Find_Vprime(G,Cprime)
    G_prime = Gprime(G, V_prime)
    
    new_node_id = find_next(V_prime, G_prime, explored)
    
    if not new_node_id:
        continue

#     new_node_id = vertices[vertices_order[node_id]+1]
#         print(new_node_id)
    cover_size = len(Cprime) + 1  
    new_cover = Cprime.union(set([new_node_id]))
#         print(new_cover)
    # get Vprime
#         print('new cover',new_cover)
    G_prime = G_prime.remove_vertex(new_node_id,inplace = True)
    V_prime = G_prime.get_vertices()
#         print('V prime',V_prime)
    if not V_prime:
        # solution found
        if cover_size < opt_num:
            opt_num = cover_size
            opt_cover = new_cover 
            print('Optimal:', opt_num)
            continue

    
    LowerBound = cover_size + approx2(G_prime)//2

    if LowerBound < opt_num:
        new_node1 = node(new_node_id, Dnode, LowerBound,1)
        heapq.heappush(pqueue, (G_prime.num_edges, new_node1))
#         print('new decision node added')
#     else:
#         print('new decision node pruned')


    if Dnode.lb < opt_num:
        new_node0 = node(new_node_id, Dnode, Dnode.lb,0)
        heapq.heappush(pqueue, (num_uncov_edges, new_node0))
#         print('new decision node added')
#     else:
#         print('new decision node pruned')

        

Optimal: 7374


KeyboardInterrupt: 

In [666]:
G = parse_edges('../DATA/karate.graph')

In [667]:
approx2(G)

22

In [22]:
## This code is working, but not very efficient, trying to improve the speed using the following cell
class node:
    def __init__(self,id, parent,lb,state):
        self.id = id
        self.parent = parent
        self.lb = lb
        self.state = state
#         self.degree = degree
    
    def __lt__(self, other):
        return self.lb > other.lb
        
# dummy = node(0,None,LB)       

n = G.num_vertices
opt_num = n
vertices = G.get_vertices()
vertices.sort(key = lambda x: -len(G.get_vertex(x).adjacent))
opt_cover = vertices
LB = approx2(G) / 2
vertices_order = {vertex: i for i, vertex in enumerate(vertices)}

num_uncov_edges0 = G.num_edges
num_uncov_edges1 = G.num_edges - G.get_vertex_degree(vertices[0])
pqueue = [(num_uncov_edges0, node(vertices[0],None,LB,0)),(num_uncov_edges1,node(vertices[0],None,LB,1))]
heapq.heapify(pqueue)
new_node0, new_node1 = None, None
while pqueue:

    num_uncov_edges,Dnode = heapq.heappop(pqueue)
    parent = Dnode.parent
    node_id = Dnode.id   
    
#     print(Dnode, new_node1, new_node0)
    if Dnode == new_node1:
#         print('a1')
        Cprime.add(new_node_id)
    elif Dnode == new_node0:
#         print('a2')
        explored.add(new_node_id)
    else:    
#         print('a3')
        Cprime = set([Dnode.id]) if Dnode.state else set()
        explored = set() if Dnode.state else set([Dnode.id])
        while parent:
#             print([parent.id, parent.state],end='')
            if parent.state:
                Cprime.add(parent.id)
            else:
                explored.add(parent.id)
            parent = parent.parent
#     print('')
#     print('Cprime',Cprime) 
    V_prime = Find_Vprime(G,Cprime)
    G_prime = Gprime(G, V_prime)
    
    new_node_id = find_next(V_prime, G_prime, explored)
    
    if not new_node_id:
        continue

#     new_node_id = vertices[vertices_order[node_id]+1]
#         print(new_node_id)
    cover_size = len(Cprime) + 1  
    new_cover = Cprime.union(set([new_node_id]))
#         print(new_cover)
    # get Vprime
#         print('new cover',new_cover)
    G_prime = G_prime.remove_vertex(new_node_id,inplace = True)
    V_prime = G_prime.get_vertices()
#         print('V prime',V_prime)
    if not V_prime:
        # solution found
        if cover_size < opt_num:
            opt_num = cover_size
            opt_cover = new_cover 
            print('Optimal:', opt_num)
            continue

    
    LowerBound = cover_size + approx2(G_prime)//2

    if LowerBound < opt_num:
        new_node1 = node(new_node_id, Dnode, LowerBound,1)
        heapq.heappush(pqueue, (G_prime.num_edges, new_node1))
#         print('new decision node added')
#     else:
#         print('new decision node pruned')


    if Dnode.lb < opt_num:
        new_node0 = node(new_node_id, Dnode, Dnode.lb,0)
        heapq.heappush(pqueue, (num_uncov_edges, new_node0))
#         print('new decision node added')
#     else:
#         print('new decision node pruned')


        

Optimal: 14


KeyboardInterrupt: 

In [14]:
def find_next(V_prime, G_prime, Explored):
    for v in sorted(V_prime, key = lambda x: - G_prime.get_vertex_degree(x)):
        if v not in Explored:
            return v
        

In [559]:
class node:
    def __init__(self,id, parent,lb):
        self.id = id
        self.parent = parent
        self.lb = lb
#         self.state = state
    
    def __lt__(self, other):
        return self.lb < other.lb
        
# dummy = node(0,None,LB)       

n = G.num_vertices
opt_num = n
vertices = G.get_vertices()
vertices.sort(key = lambda x: -len(G.get_vertex(x).adjacent))
opt_cover = vertices
# n = 15
# vertices = [34,1,33,2,3,4,6,32,24,5,25,7,9,30,8]
vertices_order = {vertex: i for vertex, i in zip(vertices,range(n))}
pqueue = [(n-1, vertices_order[vertex_id], node(vertex_id,None,LB)) for vertex_id in vertices]
heapq.heapify(pqueue)
while pqueue:
    _, _, Dnode = heapq.heappop(pqueue)
#     print('vertex_id:', Dnode.id)
    parent = Dnode.parent
    node_id = Dnode.id   
    # get Cprime
    Cprime = set([Dnode.id]) #if parent else []
    while parent:
        Cprime.add(parent.id)
        parent = parent.parent
#     print('Cprime',Cprime) 
#     print('pqueue', [item[2].id for item in pqueue])
    for i in range(vertices_order[node_id]+1, n): 
#         print(i)
        new_node_id = vertices[i]
#         print(new_node_id)
        cover_size = len(Cprime) + 1  
        new_cover = Cprime.union(set([new_node_id]))
#         print(new_cover)
        # get Vprime
#         print('new cover',new_cover)
        V_prime = Find_Vprime(G,new_cover)
#         print('V prime',V_prime)
        if not V_prime:
            # solution found
            if cover_size < opt_num:
                opt_num = cover_size
                opt_cover = new_cover 
                print('Optimal:', opt_num, opt_cover)
                continue
        else:
            
            G_prime = Gprime(V_prime)
            LowerBound = cover_size + approx2(G_prime)//2
            
            if LowerBound < opt_num:
                new_node = node(new_node_id, Dnode, LowerBound)
                heapq.heappush(pqueue, (n-cover_size, i, new_node))
#                 print('new covered added:', new_cover)
#                 print(new_node.id, 'added as a child of ', node_id)
#             else:
#                 print('new cover pruned:', new_cover)

#                 print('prune new cover with cover size =', cover_size, 'lower bound = ', LowerBound,'optimal conver cover size:', opt_num)
                        

Optimal: 18 {1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 24, 26, 28, 30, 31, 32, 33, 34}
Optimal: 17 {1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 24, 25, 30, 31, 32, 33, 34}
Optimal: 16 {32, 33, 2, 3, 4, 5, 6, 7, 8, 9, 1, 34, 14, 24, 25, 30}
Optimal: 15 {32, 33, 2, 3, 4, 5, 6, 7, 1, 9, 34, 14, 24, 25, 30}
Optimal: 14 {32, 33, 2, 3, 4, 5, 6, 7, 1, 9, 34, 24, 25, 30}


KeyboardInterrupt: 

In [517]:
Cprime = set([34,1,33,2,3,4,6,32,24,5,25,7,9,30])
sorted(Cprime)

[1, 2, 3, 4, 5, 6, 7, 9, 24, 25, 30, 32, 33, 34]

In [507]:
V_prime = Find_Vprime(G,Cprime)
G_prime = Gprime(V_prime)

In [510]:
LowerBound = len(Cprime) + approx2(G_prime)
LowerBound

14

In [422]:
G = parse_edges('../DATA/jazz.graph')
n = G.num_vertices
vertices = G.get_vertices()
opt_num = n
opt_cover = vertices
def Branch_and_Bound(cover_set,Gprime,explored,opt_num,opt_cover):
    
    v = choose_vertex(Gprime,explored)
    print('explored',explored)
    if not v:
#         print('no v found')
        return
    v_neighbors = Gprime.get_vertex(v).get_connections()

    Gprime.remove_vertex(v)
    cover_set.add(v)
    
    if not Gprime.get_vertices():
        # solution found
        print('solution found')
        if len(cover_set) < opt_num:
            opt_cover = cover_set
            opt_num = len(cover_set)
            print('optimal:', opt_num)
            
    lower_bound = len(cover_set) + approx2(Gprime) / 2
    if lower_bound < opt_num:
        print('get here')
        Branch_and_Bound(cover_set, Gprime, explored,opt_num,opt_cover)
    
    print('v_neighbors:', v_neighbors)
    for vertex in v_neighbors:
        Gprime.add_edge(v,vertex.id)
        print('graph added vertices')
        
    cover_set.remove(v)
#     explored.add(v)
    lower_bound = len(cover_set) + approx2(Gprime) / 2
    if lower_bound < opt_num:
        Branch_and_Bound(cover_set, Gprime, explored + [v],opt_num,opt_cover)
    
#     return opt_num, opt_cover
    

In [423]:
def choose_vertex(G,explored):
    for vertex in sorted(G.get_vertices(), key = lambda x: -len(G.get_vertex(x).get_connections())):
        if vertex not in explored:
            return vertex

In [683]:
import time
import sys
# from graph import graph # my own code for graph object
import heapq
from collections import deque,defaultdict
import argparse

class vertex:
    def __init__(self,node):
        self.id = node
        self.adjacent = set()

    def __str__(self):
        # for print out result
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])

    def add_neighbor(self, neighbor):
    
        self.adjacent.add(neighbor)

    def remove_neighbor(self, neighbor):
        if neighbor in self.adjacent:
            self.adjacent.remove(neighbor)

    def is_connected(self,neighbor):
        return neighbor in self.adjacent

    def get_connections(self):
        return self.adjacent



class graph:
    # unweighted undirected graph
    # can be connected or not
    def __init__(self):
        self.vert_dict = {} # vertex_id (int) : vertex
        self.num_vertices = 0
        self.num_edges = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self,node):
        self.num_vertices += 1
        new_vertex = vertex(node)
        self.vert_dict[node] = new_vertex

    def get_vertex(self,node):
        if node in self.vert_dict:
            return self.vert_dict[node]
        else:
            return None

    def add_edge(self, frm, to):
        # for new vertices
        if frm not in self.vert_dict:
            self.add_vertex(frm)
        if to not in self.vert_dict:
            self.add_vertex(to)

        if not self.vert_dict[frm].is_connected(self.vert_dict[to]):
            self.num_edges += 1

        self.vert_dict[frm].add_neighbor(self.vert_dict[to])
        self.vert_dict[to].add_neighbor(self.vert_dict[frm])


    def remove_edge(self, frm, to):
        self.vert_dict[frm].remove_neighbor(self.vert_dict[to])
        self.vert_dict[to].remove_neighbor(self.vert_dict[frm])
        self.num_edges -= 1
    
    def remove_vertex(self,node,inplace = False):
        
        G = self.copy() if not inplace else self
        
        for neighbor in list(G.vert_dict[node].get_connections()):
            G.remove_edge(node, neighbor.id)
            if not neighbor.get_connections():
                del G.vert_dict[neighbor.id]
        del G.vert_dict[node]
        
        return G
            

    def get_vertices(self):
        # return a list of ints, the id of vertices
        return list(self.vert_dict.keys())
    
    def get_vertex_degree(self,node):
        return len(self.get_vertex(node).adjacent)
    
    def get_max_degree_vertex(self):
        return max(self.vert_dict, key = lambda x: len(self.get_vertex(x).get_connections()))
    
    def copy(self):
        copy = graph()
        for v in self.vert_dict:
            for u in self.vert_dict[v].get_connections():
                copy.add_edge(v,u.id)
        return copy


        # read data and construct a graph object
def parse_edges(filename):
    # parse edges from graph file to create your graph object
    # filename: string of the filename
    f = open(filename, "r")
    n_vertices, n_edges, _ = f.readline().split(' ')
    n_vertices, n_edges = int(n_vertices), int(n_edges)

    G = graph()  # create a graph

    # add edges to the graph
    for i in range(1, n_vertices + 1):
        neighbors = f.readline().rstrip().split(' ')
        for neighbor in neighbors:
            if neighbor != '':
                if int(neighbor) > i:
                    G.add_edge(i, int(neighbor))
    return G 

def approx2(G):
    # G: a graph object
    # return: the number of vertices of a 2-approximation solution to the min vertex cover problem of G

    S = set()
    for v in G:
        if v in S:
            continue
        for neighbor in v.get_connections():
            
            if neighbor.id < v.id:
                continue
            if v.id not in S and neighbor.id not in S:
                S.add(v.id)
                S.add(neighbor.id)
    return len(S) # dividing the result by 2 gives us a lower bound of the minimum vertex cover of G


def Find_Vprime(G, Cprime):
    'Cprime: a list of numbers in {1, 2, ..., n}'
    'Return: a set of vertices Vprime of the graph Gprime'
    Vprime = set()
#     Cprime = set(Cprime)
#     n = G.num_vertices
    for node in G.get_vertices():
        if node in Cprime:
            continue
            
        for neighbor in G.get_vertex(node).adjacent:
            if neighbor.id not in Cprime:
                Vprime.add(node)
                break
    return Vprime


def Gprime(G,Vprime):
    'Construct graph Gprime from Vprime'
    G_prime = graph()
    for node in Vprime:
        for neighbor in G.get_vertex(node).adjacent:
            if neighbor.id > node and neighbor.id in Vprime:
                G_prime.add_edge(node,neighbor.id)
    return G_prime

def find_next(V_prime, G_prime, Explored):
    for v in sorted(V_prime, key = lambda x: - G_prime.get_vertex_degree(x)):
        if v not in Explored:
            return v


class node:
    'decision node'
    def __init__(self,id, parent,lb,state):

        self.id = id # the id of the vertex in the original graph G
        self.parent = parent # parent of the current decision node
        self.lb = lb # lower bound of the current decision node 
        self.state = state # state == 1 if we consider including vertex(id) in the vertex cover
    
    def __lt__(self, other): 
        return self.lb > other.lb
         

def Branch_and_Bound(G, start_time, cutoff):
    """
    Find min vertex using a branch and bound algorithm.

    Consider all possible vertex covers and prune nonpromising ones using lower bounds and upper bounds

    Input: a networkx undirected graph
    Returns: the minimum vertex cover
    """
    n = G.num_vertices   
    vertices = G.get_vertices()
    # sort vertices in degree 
    vertices.sort(key = lambda x: -len(G.get_vertex(x).adjacent)) 

    opt_cover = vertices
    opt_num = n

    # initial lowerbound
    LB = approx2(G) / 2   

    # to get the order of vertex using vertex id
    vertices_order = {vertex: i for i, vertex in enumerate(vertices)}

    # number of uncovered edges
    num_uncov_edges0 = G.num_edges # not selecting the vertex[vertices_order[0]]
    num_uncov_edges1 = G.num_edges - G.get_vertex_degree(vertices[0]) # selecting the vertex[vertices_order[0]]

    # initialize priority queue with the first two decision nodes and their corresponding number of uncovered edges
    pqueue = [(num_uncov_edges0, node(vertices[0],None,LB,0)),(num_uncov_edges1,node(vertices[0],None,LB,1))]
    heapq.heapify(pqueue)

    while pqueue and ((time.time() - start_time) < cutoff):

        # get the most promising decision node with the least number of uncovered edges, then the highest lower bound
        num_uncov_edges,Dnode = heapq.heappop(pqueue)

        parent = Dnode.parent
        node_id = Dnode.id   
            
        # initialzie cover set Cprime and explored vertices     
        Cprime = set([Dnode.id]) if Dnode.state else set() # vertices in the conver set
        explored = set() if Dnode.state else set([Dnode.id]) # vertices explored but not in the cover set

        # trace back the decision tree to find Cprime and explored
        while parent:
    
            if parent.state:
                Cprime.add(parent.id)
            else:
                explored.add(parent.id)

            parent = parent.parent
        
        # find V_prime from Cprime
        V_prime = Find_Vprime(G,Cprime)
        # construct G_prime from V_prime
        G_prime = Gprime(G, V_prime)
        
        # find the vertex with the highest degree in G_prime that has not been explored
        new_node_id = find_next(V_prime, G_prime, explored)
        

        if not new_node_id:
            continue # prune it

        
        # Branch 1: add vertex(new_node_id) to the cover set

        cover_size = len(Cprime) + 1  
        new_cover = Cprime.union(set([new_node_id]))
    
        G_prime = G_prime.remove_vertex(new_node_id,inplace = True)
        V_prime = G_prime.get_vertices()
   
        if not V_prime:
            # solution found
            if cover_size < opt_num:
                opt_num = cover_size
                opt_cover = new_cover 
                print('Optimal:', opt_num)
                continue

        
        LowerBound = cover_size + approx2(G_prime)//2

        if LowerBound < opt_num:
            new_node1 = node(new_node_id, Dnode, LowerBound,1)
            heapq.heappush(pqueue, (G_prime.num_edges, new_node1))
    

        # Branch 2: don't add vertex(new_node_id) to the cover set
        if Dnode.lb < opt_num:
            new_node0 = node(new_node_id, Dnode, Dnode.lb,0)
            heapq.heappush(pqueue, (num_uncov_edges, new_node0))

    return opt_num, opt_cover


def main(graph, algo, cutoff, seed):
    G = parse_edges(graph)
    graph_name = graph.split('/')[-1].split('.')[0]
    sol_file = "_".join([graph_name, algo, str(cutoff)]) + '.sol'
    trace_file = "_".join([graph_name, algo, str(cutoff)]) + '.trace'
    output_dir = '../output/'

    start_time = time.time()

    if algo == 'BnB':
        num_vc_nodes, vc = Branch_and_Bound(G, start_time, cutoff)


    total_time = round((time.time() - start_time), 5)
    print('BnB Algo Runtime: ' + str(total_time))

    # Create output directory if it does not exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(os.path.join(output_dir, sol_file), 'w') as f:
        f.write(str(num_vc_nodes) + "\n")
        f.write(','.join([str(n) for n in sorted(vc)]))
    f.close()

    with open(os.path.join(output_dir, trace_file), 'w') as f:
        f.write(' '.join([str(total_time), str(num_vc_nodes)]))
    f.close()

# Run as executable from terminal
# if __name__ == '__main__':
#     #parse arguments in the following format: python code/approx.py -inst DATA/jazz.graph -alg approx -time 600 -seed 30
#     parser = argparse.ArgumentParser(description='Run algorithm with specified parameters')
#     parser.add_argument('-inst', type=str, required=True, help='graph file')
#     parser.add_argument('-alg', type=str, required=True, help='algorithm to use')
#     parser.add_argument('-time', type=float, default=600, required=False, help='runtime cutoff for algorithm')
#     parser.add_argument('-seed', type=int, default=30, required=False, help='random seed for algorithm')
#     args = parser.parse_args()

#     main(args.inst, args.alg, args.time, args.seed)

        

In [684]:
main('../DATA/netscience.graph', 'BnB', 60, 30)

Optimal: 899
BnB Algo Runtime: 60.00075
